In [1]:
import sys
print(sys.executable)

import numpy as np
import shutil
import glob
import cv2
import os

colors = np.array([[239, 188, 57], # Orange
                   [198, 27, 27],  # Red
#                    [198, 27, 27],  # Red
                   [11, 147, 8], # Green
                   [80, 81, 80],# Gray
                   [0,0,0],
                  ])

/home/nathan/miniconda3/envs/milk/bin/python


In [2]:
def get_whitespace(src, threshold=190):
    if len(src.shape) == 3:
        src = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
    white = src > threshold
    gray = src == 128
    
    white = gray + white
    dims = src.shape[0] * src.shape[1]
    print(src.shape, white.shape, np.sum(white) / float(dims))
    return white

def maskout_whitespace(white_img, target, values=[0,0,0]):
    if white_img.shape[0] != target.shape[0]:
        print('Shape mismatch; resizing: {} --> {}'.format(white_img.shape, target.shape))
        white_img = cv2.resize((white_img).astype(np.uint8), dsize=(target.shape[1], target.shape[0]), 
                               interpolation=cv2.INTER_NEAREST)
        white_img = white_img.astype(np.bool)
        
    target = np.split(target, 3, axis=-1)
    
    for k in range(3):
        target[k][white_img] = 255
        
    target = np.concatenate(target, axis=-1)
    return target

In [3]:
# white_src = '/home/nathan/Dropbox/_papers/gleason_grade/colorized/validataion_wsi_colored/*colored.jpg'
white_src = '../densenet_small/ext_75pct/inference/*rgb.jpg'
white_src_imgs = sorted(glob.glob(white_src))

white_basenames = [os.path.basename(x).split('_')[0] for x in white_src_imgs]
# white_imgs = [get_whitespace(cv2.imread(x)) for x in white_src_imgs]
white_imgs = {}

for white_img, white_basename in zip(white_src_imgs, white_basenames):
    img = cv2.imread(white_img)
    white_imgs[white_basename] = img

In [4]:
mask_src = '../data/validation_svs/*.png'
mask_list = sorted(glob.glob(mask_src))
mask_basenames = [os.path.basename(x).replace('.png', '') for x in mask_list]

for mask, mask_basename in zip(mask_list, mask_basenames):
    if mask_basename not in white_basenames:
        print('{} not in reference'.format(mask_basename))
        continue
        
    print(mask)
    maskx = cv2.imread(mask, -1)
    rgb = white_imgs[mask_basename]
    
    whitemask = get_whitespace(rgb)
    rgb = np.copy(maskout_whitespace(whitemask, rgb))
    
    x,y = rgb.shape[1], rgb.shape[0]
    maskx = cv2.resize(maskx, dsize=(x,y), interpolation=cv2.INTER_NEAREST)
    
    for k in range(4):
        classx = maskx == k
        nx = np.sum(classx)
        print('\t{}: {}'.format(k, np.sum(classx)))
        if nx == 0:
            continue
        
        rgb[classx, :] = colors[k][::-1]
        
    dst = mask.replace('.png', '_recolored.jpg')
    print(dst, rgb.shape)
    cv2.imwrite(dst, rgb)

../data/validation_svs/1304700.png
(1640, 2988) (1640, 2988) 0.5719061204166258
	0: 449235
	1: 0
	2: 0
	3: 0
../data/validation_svs/1304700_recolored.jpg (1640, 2988, 3)
../data/validation_svs/1304701.png
(2090, 2988) (2090, 2988) 0.4720867521121167
	0: 44189
	1: 0
	2: 0
	3: 0
../data/validation_svs/1304701_recolored.jpg (2090, 2988, 3)
../data/validation_svs/1304702.png
(2074, 3735) (2074, 3735) 0.5798437723894614
	0: 267812
	1: 0
	2: 0
	3: 0
../data/validation_svs/1304702_recolored.jpg (2074, 3735, 3)
../data/validation_svs/1304703.png
(2287, 3610) (2287, 3610) 0.49070417280861234
	0: 31887
	1: 0
	2: 0
	3: 0
../data/validation_svs/1304703_recolored.jpg (2287, 3610, 3)
../data/validation_svs/1304704.png
(2782, 3112) (2782, 3112) 0.46399572906251907
	0: 530954
	1: 0
	2: 0
	3: 0
../data/validation_svs/1304704_recolored.jpg (2782, 3112, 3)
../data/validation_svs/1304705.png
(2749, 2739) (2749, 2739) 0.5309818924495894
	0: 378951
	1: 0
	2: 0
	3: 0
../data/validation_svs/1304705_recolored.

(1338, 2054) (1338, 2054) 0.5665983323217813
	0: 0
	1: 0
	2: 1173108
	3: 0
../data/validation_svs/1305475_recolored.jpg (1338, 2054, 3)
../data/validation_svs/1305476.png
(2883, 3486) (2883, 3486) 0.6870396207494862
	0: 0
	1: 0
	2: 233601
	3: 0
../data/validation_svs/1305476_recolored.jpg (2883, 3486, 3)
../data/validation_svs/1305477.png
(2758, 3610) (2758, 3610) 0.5203841155118627
	0: 0
	1: 0
	2: 601583
	3: 0
../data/validation_svs/1305477_recolored.jpg (2758, 3610, 3)
../data/validation_svs/1305478.png
(2341, 2614) (2341, 2614) 0.4803921120036134
	0: 0
	1: 0
	2: 1018132
	3: 0
../data/validation_svs/1305478_recolored.jpg (2341, 2614, 3)
../data/validation_svs/1305479.png
(2259, 2490) (2259, 2490) 0.4722754675185914
	0: 0
	1: 0
	2: 2461650
	3: 0
../data/validation_svs/1305479_recolored.jpg (2259, 2490, 3)
../data/validation_svs/1305480.png
(2142, 2988) (2142, 2988) 0.5280843260999178
	0: 0
	1: 0
	2: 1974169
	3: 0
../data/validation_svs/1305480_recolored.jpg (2142, 2988, 3)
../data/val